In [182]:
import numpy as np

from ppfft.tools.pad import pad
from ppfft.tools.new_fft import new_fft


# Fractional Fourier transform and its adjoint

Let $n \in \mathbb{N}$ and $c \in \mathbb{C}^n$. We denote by $n/2$ the quotient of the division of $n$ by $2$ (which corresponds to ``n//2`` in Python). We define:
$$
\mathcal{D}(n) = 
\begin{cases}
\llbracket - n/2, n/2 - 1\rrbracket &\text{ if } n \text{ is even} \\
\llbracket - n/2, n/2 \rrbracket &\text{ else}
\end{cases}
$$

Let $\beta \in \mathbb{R}$. The $\beta$-Fractional Fourier Transform of $c$ is defined as:
$$
\forall k \in \mathcal{D}(n), \, F^\alpha(c)(k) = \sum_{u \in \mathcal{D}(n)} c(u) \exp\left(-2i \pi \dfrac{\alpha k u}{n} \right)
$$

When $\alpha$ is rational, say $\alpha = \dfrac{a}{b}$, it becomes:
$$
\forall k \in \mathcal{D}(n), \, F^\alpha(c)(k) = \sum_{u \in \mathcal{D}(n)} c(u) \exp\left(-2i \pi \dfrac{a k u}{b n} \right)
$$

Using zero padding and the FFT, we can compute:
$$
\forall k' \in \mathcal{D}(nb), \, F^\alpha(c)(k') = \sum_{u \in \mathcal{D}(n)} c(u) \exp\left(-2i \pi \dfrac{k' u}{b n} \right)
$$

We then have to select the $k'$ values corresponding to the $a k$ ones.

In [183]:
def frac_fft(c: np.ndarray, a: int, b: int, axis=-1):
    """
    frac FFT with alpha = a / b
    """
    shape = np.shape(c)
    n = shape[axis]
    q_n, r_n = divmod(n, 2)
    m = b * n
    q_m, r_m = divmod(m, 2)

    aux_shape = list(shape)
    aux_shape[axis] = m
    aux_shape = tuple(aux_shape)
    aux = pad(c, aux_shape)

    indices = (a * np.arange(-q_n, q_n + r_n)) % m  # between 0 and m - 1
    indices[indices >= q_m + r_m] -= m  # between -m//2 and -1

    indices += q_m

    return np.take(new_fft(aux, axis), indices, axis=axis)


In [184]:
def naive_frac_fourier(x, alpha):
    n = len(x)

    if n % 2 == 0:
        half_n = n//2
        # from -(n//2) to n//2 - 1 : 2*(n//2) = n points
        u = np.arange(-half_n, half_n)
        k = np.arange(-half_n, half_n)

    else:
        half_n = n//2
        # from -(n//2) to n//2 : 2*(n//2) + 1 = n points
        u = np.arange(-half_n, half_n + 1)
        k = np.arange(-half_n, half_n + 1)

    ku = np.exp(-2j * np.pi * alpha * np.einsum("k,u->ku", k, u) / n)
    res = np.einsum("u,ku->k", x, ku)

    return res


In [185]:
n_list = np.arange(10, 20)
a_list = np.arange(1, 7)
b_list = np.arange(2, 6)
l = []

for n in n_list:
    for a in a_list:
        for b in b_list:
            c = np.random.rand(n)
            l.append(np.max(np.abs(frac_fft(c, a, b) - naive_frac_fourier(c, a/b))))

print(np.max(l))


3.1780315884455535e-14


In [186]:
x = np.random.rand(10, 11)

a = 3
b = 5

full = frac_fft(x, a, b)

for i in range(10):
    print(np.max(np.abs(full[i] - frac_fft(x[i], a, b))))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


## Adjoint of ``frac_fft``

In [187]:
from ppfft.tools.frac_fft import frac_fft


In [188]:
def adj_frac_fft(c, a, b, axis=-1):
    return frac_fft(c, -a, b, axis)


In [189]:
n_list = [4, 5]
a_list = np.arange(-10, 11)
b_list = [2, 3, 4]
err_list = []

for n in n_list:
    for a in a_list:
        for b in b_list:
            x = np.random.rand(n)
            y = np.random.rand(n)
            err_list.append(np.vdot(frac_fft(x, a, b), y) -
                            np.vdot(x, adj_frac_fft(y, a, b)))

print(np.max(np.abs(err_list)))


2.6645352591003757e-15


# Fast frac fft

The frac fft we defined here is actually slow due to zero-padding to size $n^2$ during the ppfft. Here, we define a general frac fft:
$$
\forall u \in \mathcal{D}(n), X(u) = \sum_{j \in \mathcal{D}(n)} x(j) \exp\left(-2i\pi \beta j u \right)
$$

In [190]:
from scipy.signal import czt

In [191]:
def true_x_right_u_right(x, beta):
    n = len(x)
    q_n, r_n = divmod(n, 2)

    j = np.arange(q_n + r_n)
    x_right = x[q_n:]
    res = []
    for u in range(q_n + r_n):
        exp = np.exp(-2j * np.pi * beta * u * j)
        res.append(np.sum(x_right * exp))

    return np.array(res)

In [192]:
def x_right_u_right(x, beta):
    n = len(x)
    q_n, _ = divmod(n, 2)
    x_right = x[q_n:]
    w = np.exp(-2j * np.pi * beta)
    return czt(x_right, w=w)


In [193]:
n = 10
x = np.random.rand(n)
beta = np.random.rand()
print(np.allclose(true_x_right_u_right(x, beta), x_right_u_right(x, beta)))

n = 11
x = np.random.rand(n)
beta = np.random.rand()
print(np.allclose(true_x_right_u_right(x, beta), x_right_u_right(x, beta)))

True
True


In [194]:
def true_x_right_u_left(x, beta):
    n = len(x)
    q_n, r_n = divmod(n, 2)

    j = np.arange(q_n + r_n)
    x_right = x[q_n:]
    res = []
    for u in range(-q_n, 0):
        exp = np.exp(-2j * np.pi * beta * u * j)
        res.append(np.sum(x_right * exp))

    return np.array(res)

In [195]:
def x_right_u_left(x, beta):
    n = len(x)
    q_n, _ = divmod(n, 2)
    x_right = x[q_n:]
    w = np.exp(2j * np.pi * beta)
    a = np.exp(-2j * np.pi * beta)
    return czt(x_right, m=q_n, w=w, a=a)[::-1]

In [196]:
n = 10
x = np.random.rand(n)
beta = np.random.rand()
print(np.allclose(true_x_right_u_left(x, beta), x_right_u_left(x, beta)))

n = 11
x = np.random.rand(n)
beta = np.random.rand()
print(np.allclose(true_x_right_u_left(x, beta), x_right_u_left(x, beta)))

True
True


In [197]:
def true_x_left_u_right(x, beta):
    n = len(x)
    q_n, r_n = divmod(n, 2)

    j = np.arange(-q_n, 0)
    x_left = x[:q_n]
    res = []
    for u in range(q_n + r_n):
        exp = np.exp(-2j * np.pi * beta * u * j)
        res.append(np.sum(x_left * exp))

    return np.array(res)

In [198]:
def x_left_u_right(x, beta):
    n = len(x)
    q_n, r_n = divmod(n, 2)
    x_left = x[:q_n]
    u = np.arange(q_n + r_n)
    w = np.exp(2j * np.pi * beta)
    return czt(x_left[::-1], m=q_n+r_n, w=w) * np.exp(2j * np.pi * beta * u)

In [199]:
n = 10
x = np.random.rand(n)
beta = np.random.rand()
print(np.allclose(true_x_left_u_right(x, beta), x_left_u_right(x, beta)))

n = 11
x = np.random.rand(n)
beta = np.random.rand()
print(np.allclose(true_x_left_u_right(x, beta), x_left_u_right(x, beta)))

True
True


In [200]:
def true_x_left_u_left(x, beta):
    n = len(x)
    q_n, _ = divmod(n, 2)

    j = np.arange(-q_n, 0)
    x_left = x[:q_n]
    res = []
    for u in range(-q_n, 0):
        exp = np.exp(-2j * np.pi * beta * u * j)
        res.append(np.sum(x_left * exp))

    return np.array(res)

In [201]:
def x_left_u_left(x, beta):
    n = len(x)
    q_n, _ = divmod(n, 2)

    x_left = x[:q_n]
    u = np.arange(-q_n, 0)
    w = np.exp(-2j * np.pi * beta)
    a = np.exp(2j * np.pi * beta)
    return czt(x_left[::-1], w=w, a=a)[::-1] * np.exp(2j * np.pi * beta * u)

In [202]:
n = 10
x = np.random.rand(n)
beta = np.random.rand()
print(np.allclose(true_x_left_u_left(x, beta), x_left_u_left(x, beta)))

n = 11
x = np.random.rand(n)
beta = np.random.rand()
print(np.allclose(true_x_left_u_left(x, beta), x_left_u_left(x, beta)))

True
True


In [203]:
def fast_frac_fft(x, beta):
    left_1 = x_left_u_right(x, beta)
    left_2 = x_left_u_left(x, beta)
    right_1 = x_right_u_right(x, beta)
    right_2 = x_right_u_left(x, beta)

    return np.concatenate((left_2 + right_2, left_1 + right_1))

def test_fast_frac_fft(x, beta):
    n = len(x)
    q_n, r_n = divmod(n, 2)
    us = np.arange(-q_n, q_n + r_n)
    js = np.arange(-q_n, q_n + r_n)

    res = []
    for u in us:
        res.append(np.sum(x * np.exp(-2j * np.pi * beta * u * js)))

    return np.array(res)

In [204]:
n = 1000
x = np.random.rand(n)
beta = np.random.rand()
np.allclose(fast_frac_fft(x, beta), test_fast_frac_fft(x, beta))

True

In [205]:
n = 1001
x = np.random.rand(n)
beta = np.random.rand()
np.allclose(fast_frac_fft(x, beta), test_fast_frac_fft(x, beta))

True

# For the forward ppfft

For the forward ppfft, we actually need to output $n + 1$ points when the input has size $n$. We want to compute:
$$
\forall u \in \mathcal{D}(n+1), \sum_{j \in \mathcal{D}(n)} x(j) \exp\left(-2i\pi \frac{\alpha j u}{n} \right)
$$

We can see that this is simply the frac_fft we defined above, applied to a *carefully* zero-padded $x$.

In [206]:
def frac_fft_for_ppfft(x, alpha):
    n = len(x)
    if n % 2 == 0:
        x_pad = np.pad(x, (0, 1))
    else:
        x_pad = np.pad(x, (1, 0))
    return fast_frac_fft(x_pad, alpha / n)

In [207]:
def test_frac_fft_for_ppfft(x, alpha):
    n = len(x)
    q_n, r_n = divmod(n, 2)
    q, r = divmod(n + 1, 2)
    us = np.arange(-q, q + r)
    js = np.arange(-q_n, q_n + r_n)

    res = []
    for u in us:
        res.append(np.sum(x * np.exp(-2j * np.pi * alpha * u * js / n)))

    return np.array(res)

In [208]:
n = 10
x = np.random.rand(n)
alpha = np.random.rand()
np.allclose(test_frac_fft_for_ppfft(x, alpha), frac_fft_for_ppfft(x, alpha))


True

In [209]:
n = 11
x = np.random.rand(n)
alpha = np.random.rand()
np.allclose(test_frac_fft_for_ppfft(x, alpha), frac_fft_for_ppfft(x, alpha))

True

# Fast adjoint

In [210]:
from ppfft.tools.frac_fft import frac_fft_for_ppfft

In [211]:
def adj_frac_fft_for_ppfft(y, alpha):
    n = len(y) - 1
    if n % 2 == 0:
        return fast_frac_fft(y, -alpha / n)[:-1]
    else:
        return fast_frac_fft(y, -alpha / n)[1:]

In [212]:
n = 10
x = np.random.rand(n)
y = np.random.rand(n + 1)
alpha = np.random.rand()

np.isclose(np.vdot(frac_fft_for_ppfft(x, alpha), y), np.vdot(x, adj_frac_fft_for_ppfft(y, alpha)))

True

In [213]:
n = 11
x = np.random.rand(n)
y = np.random.rand(n + 1)
alpha = np.random.rand()

np.isclose(np.vdot(frac_fft_for_ppfft(x, alpha), y), np.vdot(x, adj_frac_fft_for_ppfft(y, alpha)))

True